# Summarize relative astrometry for each band and compare between all bands

(only for systems that have all 3 bands modelled)

In [1]:
import pickle
import numpy as np
import h5py

# define systems and filters
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filters = ['F475X', 'F814W', 'F160W']

for name in names:
    print(f"\n{name}")
    image_separations = {}
    image1_dists = {}
    image2_dists = {}

    for filt in filters:
        filename = f"../cutout_data/{name}/{filt}/{name}_{filt}_final.hdf5"

        try:
            with h5py.File(filename, "r") as f:
                kwargs = {}
                for key in f:
                    kwargs[key] = f[key][()]

            x1, y1 = kwargs["image_position"][0][0], kwargs["image_position"][1][0]
            x2, y2 = kwargs["image_position"][0][1], kwargs["image_position"][1][1]

            # use different keys for F160W lens center since it has two lens light profiles
            # the lens light centers are always joined, so one can choose either for this analysis
            if filt == "F160W":
                ra_lens = kwargs['center_x_0']
                dec_lens = kwargs['center_y_0']
            else:
                ra_lens = kwargs['center_x']
                dec_lens = kwargs['center_y']

            # image separation
            sep = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            image_separations[filt] = sep
            print(f"  {filt} separation: {sep:.4f}\"")

            # distances from lens center
            dist1 = np.sqrt((x1 - ra_lens)**2 + (y1 - dec_lens)**2)
            dist2 = np.sqrt((x2 - ra_lens)**2 + (y2 - dec_lens)**2)
            image1_dists[filt] = dist1
            image2_dists[filt] = dist2

            print(f"    Distance to Image 1: {dist1:.4f}\"")
            print(f"    Distance to Image 2: {dist2:.4f}\"")

        except Exception as e:
            image_separations[filt] = None
            image1_dists[filt] = None
            image2_dists[filt] = None
            print(f"{filt} separation: not available")

    # summary of differences
    sep_values = [v for v in image_separations.values()]
    
    max_diff = max(sep_values) - min(sep_values)
    print(f"  Max difference in separation: {max_diff:.4f}\"")

    dist1_values = [v for v in image1_dists.values()]
    dist2_values = [v for v in image2_dists.values()]

    d1_diff = max(dist1_values) - min(dist1_values)
    print(f"  Max difference in distance to Image 1: {d1_diff:.4f}\"")

    d2_diff = max(dist2_values) - min(dist2_values)
    print(f"  Max difference in distance to Image 2: {d2_diff:.4f}\"")




J0407-5006
  F475X separation: 1.7209"
    Distance to Image 1: 0.3113"
    Distance to Image 2: 1.4096"
  F814W separation: 1.7207"
    Distance to Image 1: 0.3018"
    Distance to Image 2: 1.4189"
  F160W separation: 1.6496"
    Distance to Image 1: 0.2137"
    Distance to Image 2: 1.4364"
  Max difference in separation: 0.0713"
  Max difference in distance to Image 1: 0.0976"
  Max difference in distance to Image 2: 0.0268"

J0806+2006
  F475X separation: 1.4858"
    Distance to Image 1: 1.0382"
    Distance to Image 2: 0.4477"
  F814W separation: 1.4865"
    Distance to Image 1: 1.0554"
    Distance to Image 2: 0.4316"
  F160W separation: 1.4760"
    Distance to Image 1: 1.0515"
    Distance to Image 2: 0.4262"
  Max difference in separation: 0.0105"
  Max difference in distance to Image 1: 0.0171"
  Max difference in distance to Image 2: 0.0215"

J1442+4055
  F475X separation: 2.1316"
    Distance to Image 1: 1.3580"
    Distance to Image 2: 0.7759"
  F814W separation: 2.1319"
  

# Print coordinates of all objects

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

names = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1001+5027', 'J1442+4055',
         'J1515+1511', 'J1620+1203', 'J2325-5229']

for name in names:
    filename = f'../cutout_data/{name}/F814W/{name}_F814W_cutout.fits'
    hdul = fits.open(filename)
    header = hdul[0].header
    
    ra_deg = header['RA_TARG']
    dec_deg = header['DEC_TARG']
    
    # Create a SkyCoord object
    coord = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg)
    
    # Print in hh:mm:ss, dd:mm:ss format
    print(f"{name}: RA = {coord.ra.to_string(unit=u.hour, sep=':')}, DEC = {coord.dec.to_string(unit=u.degree, sep=':')}")
